# Apache Beam - Windowing

Windowing is the ability to group together input data into distinct collections of data based upon time.  Every element in a PCollection has an explicit or implicit time value associated with it.

We have previously seen the concept of an unbounded PCollection.  This is a PCollection that has a potentially infinite number of elements.  Consider applying an unbounded PCollection to a transform such as GroupByKey or Combine.  These transforms work on complete PCollections.  For example, GroupByKey wants to examine *every* element in a PCollection and place it in a group.  How can that be done if there are always *more* elements that could arrive?  The answer is to perform the transform on a set of elements and not *all* the possible elements.  The way we build that set of elements is based on time.  For example, we could create a set of all elements that have arrived in the last hour.  This would be a bounded set of elements.

The entire logical PCollection is thus split into multiple sets based on the concept of a window of time.

There is always a default window called the Global Window.  This is the span of time from -ve Infinity to +ve Infinity (i.e. all time).

Windowing also has concepts called:

* Watermarks
* Triggers
* Panes
* Late arrivals

Windowing works with primitives that are windowing aware.  These include `GroupByKey` and `Combine`.

When we think about data arriving as input into Beam, each element of data has an associated timestamp.  We call this the *event* time.  It is the time that the event is considered to have actually happened.  For example, consider a purchase in a store.  There was a moment in time where the purchase happened.  We may then end up with a record such as:

```
Sale(item="Blue", amount=12.0, time="2022-12-21T13:30:01Z")
```

Now consider this record being processed by beam.  It will be processed at some time in the future.  We call this the *processing* time.  We now have *two* concepts of time.  One being event time which is when the event is said to have happened and the other being processing time which is when we are processing the event.

By considering these two notions of time, we can form some conclusions.  The first is that processing time always has to be later than event time for a given element.  Hopefully this is obvious.  We can't process an event before the event has happened.

Our ideal situation is one where the difference between processing time and event time is as small as possible.  In English, we would say that the data is processed as soon after its creation as possible.  However, it is usually not the case that the this difference (which we call lag) is zero.  Instead, there will be a delay between the event occurring and the event being processed.  For example, if it takes 100ms for an event to transition across a network, then the processing time will be no less than 100ms after event time.

Now imagine a magical function which we will call *watermark*.   This function takes as input an instant in event time and returns an instant in processing time such that the claim is that we will have received all possible events prior to this event time at the given processing time.  This becomes useful to us when we want to process groups of events.  Let us imagine sales records for our 24x7 online stores.  We want to aggregate all these sales records to produce a daily total.  A simple solution would be to wait till midnight (00:00:00) and then sum up all the records for the last 24 hours.  However, consider what we have just been discussing.  If we did our summation at exactly midnight, we would be missing records that have not yet been received but have actually happened.  If we assumed a latency of 5 minutes, then we shouldn't do our summation until at least 5 past midnight (00:05:00) or we are in danger of not including all the data.  Generically, the time we should start our processing is:

```
Time<processing> = watermark(Time<event>)
```

or for our example:

```
00:05:00 = watermark(00:00:00)
```

For further consideration, we can now classify messages into one of three categories.

* A message received by beam < `watermark(Time<event>)` is considered *early*
* A message received by beam = `watermark(Time<event>)` is considered *on time*
* A message received by beam > `watermark(Time<event>)` is considered *late*

If we are processing data that doesn't do any aggregations (eg. groupings or combines) then event time, processing time or late data doesn't come into play.  However, if we are doing aggregations, we need to consider the concepts of event time, processing time and late data.

See also:
* [Programming Guide: Windowing](https://beam.apache.org/documentation/programming-guide/#windowing)
* [JavaDoc: Window](https://beam.apache.org/releases/javadoc/2.43.0/org/apache/beam/sdk/transforms/windowing/Window.html)
* [JavaDoc: FixedWindows](https://beam.apache.org/releases/javadoc/2.43.0/org/apache/beam/sdk/transforms/windowing/FixedWindows.html)
* [Video: How to process stream data on Apache Beam](https://www.youtube.com/watch?v=oJ-LueBvOcM)
* [Video: Understanding exactly-once processing and windowing in streaming pipelines](https://www.youtube.com/watch?v=DraQGkARegE)
* [Testing Unbounded Pipelines in Apache Beam](https://beam.apache.org/blog/test-stream/)
* [Streaming 101: The world beyond batch](https://www.oreilly.com/radar/the-world-beyond-batch-streaming-101/)
* [Streaming 102: The world beyond batch](https://www.oreilly.com/radar/the-world-beyond-batch-streaming-102/)
* [Book: Streaming Systems](http://streamingsystems.net/)

In [54]:
%%loadFromPOM

<dependency>
  <groupId>org.apache.beam</groupId>
  <artifactId>beam-sdks-java-core</artifactId>
  <version>2.43.0</version>
</dependency>

<dependency>
  <groupId>org.apache.beam</groupId>
  <artifactId>beam-runners-direct-java</artifactId>
  <version>2.43.0</version>
  <scope>runtime</scope>
</dependency>

<dependency>
  <groupId>org.slf4j</groupId>
  <artifactId>slf4j-api</artifactId>
  <version>2.0.6</version>
</dependency>

Next we define our imports required for execution.

In [45]:
import java.util.Arrays;
import java.util.List;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.options.Default;
import org.apache.beam.sdk.options.Description;
import org.apache.beam.sdk.options.PipelineOptionsFactory;
import org.apache.beam.sdk.options.PipelineOptions;
import org.apache.beam.sdk.options.StreamingOptions;
import org.apache.beam.sdk.transforms.Create;
import org.apache.beam.sdk.transforms.GroupByKey;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.transforms.join.CoGbkResult;
import org.apache.beam.sdk.transforms.join.KeyedPCollectionTuple;
import org.apache.beam.sdk.transforms.join.CoGroupByKey;
import org.apache.beam.sdk.transforms.Combine.CombineFn;
import org.apache.beam.sdk.transforms.Combine;
import org.apache.beam.sdk.transforms.SerializableFunction;
import org.apache.beam.sdk.transforms.Sum;
import org.apache.beam.sdk.transforms.Flatten;
import org.apache.beam.sdk.transforms.windowing.Window;
import org.apache.beam.sdk.transforms.windowing.FixedWindows;
import org.apache.beam.sdk.transforms.windowing.AfterWatermark;
import org.apache.beam.sdk.transforms.Count;
import org.apache.beam.sdk.transforms.WithTimestamps;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.PCollectionList;
import org.apache.beam.sdk.values.KV;
import org.apache.beam.sdk.values.PDone;
import org.apache.beam.sdk.values.TupleTag;
import org.apache.beam.sdk.values.TypeDescriptor;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.coders.KvCoder;
import org.apache.beam.sdk.coders.StringUtf8Coder;
import org.apache.beam.sdk.schemas.annotations.DefaultSchema;
import org.apache.beam.sdk.schemas.JavaBeanSchema;

//import java.time.Instant;
import org.joda.time.Instant;
import org.joda.time.Duration;


String args[] = new String[] {};
var options = PipelineOptionsFactory.fromArgs(args).withValidation().create();

Lets look at time stamps on elements.  Each element has a timestamp.  In a ParDo, we can access (and log) the timestamp using the `ProcessContext.getTime()`.  This returns an `org.joda.time.Instant` object.  Should we wish, we can also set the value of the timestamp on an element by calling `OutputReceiver.outputWithTimestamp`.  This could be useful if we have a `PCollection` of elements where the timestamp value is a property of the element data and not implicit in the element itself.  However, a better was is to use the pre-build transform called `WithTimestamps`.  This transform takes a `SerializableFunction` which receives elements as input and returns `Instant` (timestamp) values that are then associated with the elements.

Some of the transforms that are source IO produce timestamps implicitly.  For example:

* `PubsubIO` using `.withTimestampAttribute(...)` - Set the event timestamp from a named attribute.

See also:
* [JavaDoc: Class WithTimestamps](https://beam.apache.org/releases/javadoc/2.43.0/org/apache/beam/sdk/transforms/WithTimestamps.html)

In [53]:
/*
 * Execute a ParDo over the input PCollection<Sale> and for each of the Sale elements, set
 * the timestamp from the time property of the sale itself.
 */
public class Sale implements Serializable{
  private String item;
  private Double amount;
  private Instant time;
  
  public Sale(String item, Double amount, Instant time) {
    this.item = item;
    this.amount = amount;
    this.time = time;
  }
  
  public String getItem() { return item; }
  public Double getAmount() { return amount; }
  public Instant getTime() { return time; }
  public String toString() {
    return "item: " + item + ", amount: " + amount + ", time: " + time;
  }
} // Sale

public class LoggingDoFn<T> extends DoFn<T, T>  {
  @ProcessElement
  public void processElement(@Element T element, OutputReceiver<T> out, ProcessContext context) {
    System.out.println(element + ", timestamp:" + context.timestamp() + ", pane: " + context.pane());
    out.output(element);
  }
} // LoggingDoFn

var pipeline = Pipeline.create(options);
pipeline
  // Create the elements
  .apply("Create elements", Create.of(
    new Sale("blue", 10.0, Instant.parse("2022-12-11")),
    new Sale("red", 15.0, Instant.parse("2022-12-12"))
  ))
  
  /*
  // Set the timestamps on the PCollection elements from the sale.time field.
  .apply(WithTimestamps.of(new SerializableFunction<Sale, Instant>() {
    public Instant apply(Sale sale) {
      return sale.getTime();
    }
  }))
  */
  
  // Here we set the timestamps of the elements using a lambda function.
  .apply(WithTimestamps.of(sale -> sale.getTime()))
  
  // Log the elements to the output
  .apply("Print elements", ParDo.of(new LoggingDoFn<>()));
  
pipeline.run().waitUntilFinish();

item: blue, amount: 10.0, time: 2022-12-11T00:00:00.000Z, timestamp:2022-12-11T00:00:00.000Z, pane: PaneInfo.NO_FIRING
item: red, amount: 15.0, time: 2022-12-12T00:00:00.000Z, timestamp:2022-12-12T00:00:00.000Z, pane: PaneInfo.NO_FIRING


DONE

Let us set ourselves a puzzle to count the value of sales by day.

In [50]:
var pipeline = Pipeline.create(options);
pipeline
  .apply("Create elements", Create.of(
    new Sale("blue",   10.0, Instant.parse("2022-12-11")),
    new Sale("green",  10.0, Instant.parse("2022-12-11")),    
    new Sale("red",    15.0, Instant.parse("2022-12-12")),
    new Sale("yellow", 15.0, Instant.parse("2022-12-13"))
  )) 
  .apply("Set timestamps", ParDo.of(new SaleTimestampDoFn()))
  
  .apply("Window", Window
    .<Sale>into(FixedWindows.of(Duration.standardDays(1)))
    .triggering(AfterWatermark.pastEndOfWindow())
    .withAllowedLateness(Duration.ZERO)
    .discardingFiredPanes()
  )
  
  
  .apply("Get Amounts", MapElements.into(TypeDescriptors.doubles()).via(new SerializableFunction<Sale, Double>(){
    public Double apply(Sale sale) {
      return sale.getAmount();
    }
  }))
  .apply("Sum", Sum.doublesGlobally())
  .apply("Print elements", ParDo.of(new LoggingDoFn<>()));
  
pipeline.run().waitUntilFinish();

EvalException: Default values are not supported in Combine.globally() if the input PCollection is not windowed by GlobalWindows. Instead, use Combine.globally().withoutDefaults() to output an empty PCollection if the input PCollection is empty, or Combine.globally().asSingletonView() to get the default output of the CombineFn if the input PCollection is empty.